In [116]:
from dumbnano import dumbnano
#Reload the module
import importlib
importlib.reload(dumbnano)

<module 'dumbnano.dumbnano' from '/home/raingel/桌面/OOP_v2/dumbnano/dumbnano.py'>

In [117]:
dumb = dumbnano.NanoAmpliParser()

In [118]:
s = dumb.blast_2(src="./data/5_assemble/",
             des="./data/5_assemble/",
             name = "blast.csv",
             funguild=True,
             startswith ="con_"
             )

Blasting 0 to 10 of 14
BLASTING 10 sequences
Query 32S8JKDR01N submitted.
You can check the status at https://blast.ncbi.nlm.nih.gov/blast/Blast.cgi?CMD=Get&FORMAT_OBJECT=SearchInfo&RID=32S8JKDR01N
And results here: https://blast.ncbi.nlm.nih.gov/blast/Blast.cgi?CMD=Get&RID=32S8JKDR01N
Searching...
Search complete, retrieving results...
Blasting 10 to 20 of 14
BLASTING 4 sequences
Query 32SA4FUB01N submitted.
You can check the status at https://blast.ncbi.nlm.nih.gov/blast/Blast.cgi?CMD=Get&FORMAT_OBJECT=SearchInfo&RID=32SA4FUB01N
And results here: https://blast.ncbi.nlm.nih.gov/blast/Blast.cgi?CMD=Get&RID=32SA4FUB01N
Searching...
Search complete, retrieving results...


In [114]:
s

,seq,length,acc,hit_seq,hit_def,org,taxa
title,,,,,,,
2109_cluster_0_r285.fas,gaacgctagctatatgcttaacacatgcaagtcgaacgttgttttc...,1850,NC_053927,GAACGCTAGCTATATGCTTAACACATGCAAGTCGAACGTTGTTTTC...,"Ageratum conyzoides mitochondrion, complete ge...",Ageratum conyzoides,Eukaryota; Viridiplantae; Streptophyta; Embryo...
2110_cluster_0_r2154.fas,aacccccacaaaaacaaaaaaaaaacacaaaaaaaacaccacaaaa...,461,NaN,NaN,NaN,NaN,NaN
2110_cluster_1_r1902.fas,agcaagtgttggattcaaagccggtgttaaagattataaattgact...,462,NC_046893,AGCAAGTGTTGGATTCAAAGCCGGTGTTAAAGATTATAAATTGACT...,Bidens hillebrandiana subsp. polycephala chlor...,Bidens hillebrandiana subsp. polycephala,Eukaryota; Viridiplantae; Streptophyta; Embryo...
2111_cluster_0_r63.fas,agcatcgtcacaggacaacacaagtgggtttacaaggagtcaagcc...,299,KY968830,AGCATCGTCACAGGACAACACAAGTGGGTTTACAAGGAGTCAAGCC...,"Bidens alba voucher Li ZY, Jin XH, Xu SZ 13342...",Bidens alba,Eukaryota; Viridiplantae; Streptophyta; Embryo...
2111_cluster_2_r155.fas,ccacaaaaacaacccaacaacccccccaacccaacacccccacccc...,303,NaN,NaN,NaN,NaN,NaN
2110_cluster_-1_r47.fas,agcaagtgttggattcaaagccggtgttaaagattataaattgact...,645,NC_046893,AGCAAGTGTTGGATTCAAAGCCGGTGTTAAAGATTATAAATTGACT...,Bidens hillebrandiana subsp. polycephala chlor...,Bidens hillebrandiana subsp. polycephala,Eukaryota; Viridiplantae; Streptophyta; Embryo...
2440_cluster_1_r26.fas,actccagtcactagccctgccttcggcatccccctccttgcggtta...,1407,MN527993,ACTCCAGTCACTAGCCCTGCCTTCGGCATCCCCCTCCTTGCGGTTA...,"Eleutherococcus brachypus chloroplast, complet...",Eleutherococcus brachypus,Eukaryota; Viridiplantae; Streptophyta; Embryo...
2111_cluster_1_r185.fas,gaatcccgtgaaccatcgagtttttgaacgcaagttgcgcccgaag...,305,MH768095,GAATCCCGTGAACCATCGAGTTTTTGAACGCAAGTTGCGCCCGAAG...,Bidens pilosa voucher LSC107 internal transcri...,Bidens pilosa,Eukaryota; Viridiplantae; Streptophyta; Embryo...
2109_cluster_2_r274.fas,aaaacacaaccaacacccaccccaacaccacaaaccacacccccac...,1411,NaN,NaN,NaN,NaN,NaN


In [ ]:
#show scipy version
!pip show scip